## Deliverable 2. Create a Customer Travel Destinations Map.

In [89]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [90]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Talara,PE,-4.5772,-81.2719,87.8,48,0,14.97,clear sky
1,1,Yeppoon,AU,-23.1333,150.7333,68.0,64,0,13.04,clear sky
2,2,Kapaa,US,22.0752,-159.3190,66.2,82,90,6.91,overcast clouds
3,3,Bluff,NZ,-46.6000,168.3333,48.0,97,54,1.99,broken clouds
4,4,Ushuaia,AR,-54.8000,-68.3000,48.2,66,20,4.61,few clouds


In [91]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [93]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp']<=max_temp) & (city_data_df['Max Temp']>=min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Talara,PE,-4.5772,-81.2719,87.80,48,0,14.97,clear sky
5,5,Lorengau,PG,-2.0226,147.2712,80.87,83,96,13.73,overcast clouds
13,13,Meulaboh,ID,4.1363,96.1285,77.70,86,95,3.29,overcast clouds
17,17,San Lorenzo,PY,-25.3333,-57.5333,75.20,88,75,13.80,broken clouds
22,22,Saldanha,ZA,-33.0117,17.9442,71.60,60,99,12.66,overcast clouds


In [94]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                4
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [98]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_preferred_cities_df = preferred_cities_df.dropna() 
clean_preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Talara,PE,-4.5772,-81.2719,87.80,48,0,14.97,clear sky
5,5,Lorengau,PG,-2.0226,147.2712,80.87,83,96,13.73,overcast clouds
13,13,Meulaboh,ID,4.1363,96.1285,77.70,86,95,3.29,overcast clouds
17,17,San Lorenzo,PY,-25.3333,-57.5333,75.20,88,75,13.80,broken clouds
22,22,Saldanha,ZA,-33.0117,17.9442,71.60,60,99,12.66,overcast clouds


In [99]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Talara,PE,87.80,clear sky,-4.5772,-81.2719,
5,Lorengau,PG,80.87,overcast clouds,-2.0226,147.2712,
13,Meulaboh,ID,77.70,overcast clouds,4.1363,96.1285,
17,San Lorenzo,PY,75.20,broken clouds,-25.3333,-57.5333,
22,Saldanha,ZA,71.60,overcast clouds,-33.0117,17.9442,
25,Galle,LK,80.67,broken clouds,6.0367,80.2170,
31,Rikitea,PF,78.42,clear sky,-23.1203,-134.9692,
32,Kudahuvadhoo,MV,83.68,broken clouds,2.6708,72.8944,
35,Maceio,BR,77.00,mist,-9.6658,-35.7353,
37,Puerto Ayora,EC,80.60,few clouds,-0.7393,-90.3518,


In [101]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    params["location"] = f'{lat},{lng}'

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels_response = requests.get(base_url,params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels_response["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [137]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Talara,PE,87.80,clear sky,-4.5772,-81.2719,Hostal Oro Negro
5,Lorengau,PG,80.87,overcast clouds,-2.0226,147.2712,Lorengau Harbourside Hotel
13,Meulaboh,ID,77.70,overcast clouds,4.1363,96.1285,Kost Pria
17,San Lorenzo,PY,75.20,broken clouds,-25.3333,-57.5333,Hotel del Sur
22,Saldanha,ZA,71.60,overcast clouds,-33.0117,17.9442,Blue Bay Lodge
...,...,...,...,...,...,...,...
694,Sulangan,PH,81.84,overcast clouds,11.1373,123.7228,Maia's Beach Resort
696,Huarmey,PE,74.07,broken clouds,-10.0681,-78.1522,Hotel Miramar
697,Mateare,NI,87.80,scattered clouds,12.2351,-86.4281,Lotinica prados las mercedes
701,Horn Lake,US,70.00,clear sky,34.9554,-90.0348,"Hilton Garden Inn Memphis/Southaven, MS"


In [138]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
hotel_df['Hotel Name'] = hotel_df['Hotel Name'].replace('', np.nan)
clean_hotel_df = hotel_df.dropna(axis=0, subset=['Hotel Name'])
clean_hotel_df.isna().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [139]:
# TEST
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Talara,PE,87.80,clear sky,-4.5772,-81.2719,Hostal Oro Negro
5,Lorengau,PG,80.87,overcast clouds,-2.0226,147.2712,Lorengau Harbourside Hotel
13,Meulaboh,ID,77.70,overcast clouds,4.1363,96.1285,Kost Pria
17,San Lorenzo,PY,75.20,broken clouds,-25.3333,-57.5333,Hotel del Sur
22,Saldanha,ZA,71.60,overcast clouds,-33.0117,17.9442,Blue Bay Lodge
...,...,...,...,...,...,...,...
694,Sulangan,PH,81.84,overcast clouds,11.1373,123.7228,Maia's Beach Resort
696,Huarmey,PE,74.07,broken clouds,-10.0681,-78.1522,Hotel Miramar
697,Mateare,NI,87.80,scattered clouds,12.2351,-86.4281,Lotinica prados las mercedes
701,Horn Lake,US,70.00,clear sky,34.9554,-90.0348,"Hilton Garden Inn Memphis/Southaven, MS"


In [140]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [156]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt><b>Hotel Name</b></dt>
<ddstyle="text-align:left;">{Hotel Name}</dd>
<dt><b>City</b></dt>
<ddstyle="text-align:left;">{City}</dd>
<dt><b>Country</b></dt>
<ddstyle="text-align:left;">{Country}</dd>
<dt><b>Current Weather</b></dt>
<ddstyle="text-align:left;">{Current Description}</dd> and <ddstyle="text-align:left;">{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [160]:
import matplotlib.pyplot as plt
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))